In [ ]:
import sys
from _Model.Models import *
from _Utility.dataplotter import *
from _Utility.dataloader import *

import os
from os import listdir
from os.path import join
from tensorflow.python.client import device_lib
import datetime

os.environ["CUDA_VISIBLE_DEVICES"]="-1" 
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.list_physical_devices('CPU')))
print(device_lib.list_local_devices())


# Generate Dataset

In [ ]:
#periodical stationary input
stage = 250
ntotal = 2000
x_noise = 0.02
y_noise = 0.01
perturb_x = 2*x_noise*np.random.rand(2*ntotal,1) - x_noise
perturb_y = 2*y_noise*np.random.rand(2*ntotal,1) - y_noise
x0 = np.tile(np.linspace(0,1,stage),(int(2*ntotal/stage),1)).flatten()[:,None]
x  = x0[:2000] + perturb_x[:2000]
x_ext = x0 + perturb_x

#output with different trends
y0 = np.array(range(-ntotal,ntotal,1))[:,None]/ntotal
y1 = np.sin(2*np.pi*x_ext) + perturb_y # horizontal line (stationary)
# y1 = - y0 + 0.1*np.sin(2*np.pi*x_ext) + perturb_y # straight line
# y1 = -np.exp(1.5*y0+0.) + 0.1*np.sin(2*np.pi*x_ext)  + perturb_y  # divergent exponential function
# y1 = 0.3*np.exp(-1.5*y0+0.) + perturb_y + 0.1*np.sin(2*np.pi*x_ext)  # convergent exponential function
y  = (y1[:ntotal]-y1[:ntotal].min())/(y1[:ntotal].max()-y1[:ntotal].min())# + perturb_y[:2000]
y_ext = (y1-y1[:ntotal].min())/(y1[:ntotal].max()-y1[:ntotal].min())# + perturb_y
plot2(x0,1,1)
plot2(np.sin(2*np.pi*x_ext),1,1)
plot2(y,1,1)
plot2(y_ext,1,1)
plot2(np.diff(y_ext,axis=0),1,1)

In [ ]:
nfeature = 1
ncontrol = 1
twindow = 6
twindow2 = 104 # future window size
frequency = 1
trainwindow = 1500
predictwindow = ntotal-trainwindow # prediction window

# process data to generate batch data
history, control, yout, historyt, controlt, youtt, traintesthistory, traintestcontrol, obs, initialwindow, xin, yin = GenerateSets(
    x, y, frequency, twindow, twindow2, predictwindow)
historyt = historyt[:,:,-nfeature:]
history = history[:,:,-nfeature:]
print(historyt.shape, controlt.shape, youtt.shape)
print(history.shape, control.shape, yout.shape)

nbsize = 1
nepochs = 10
verbose = 1
shuffle = True
ntrain = trainwindow
nvalid = 50

# training set
xtrain = [controlt, historyt[:,-1,:]]
ytrain = youtt

# validation set
x_val = [control[ntrain:ntrain+nvalid,:,:], 
         history[ntrain:ntrain+nvalid,-1,:]]
y_val = yout[ntrain:ntrain+nvalid,:,:]

print(xtrain[0].shape, xtrain[1].shape, ytrain.shape)
print(x_val[0].shape, x_val[1].shape, y_val.shape)

# Training

In [ ]:
lr = 35*1e-4
nepochs=35
# model_long = build_phy_GRU2(nfeature, ncontrol, lr=lr) # phy-GRU 
model_long = build_phy_RNN2(nfeature, ncontrol, lr=lr) # phy-RNN 
h = model_long.fit(xtrain, youtt, 
                           validation_data=(x_val, y_val),
                           batch_size=nbsize, epochs=nepochs, 
                           verbose=verbose, shuffle=shuffle)
model_long.summary()

# Visualization

In [ ]:
nrows = 1
ncols = 1
ypred_cumsumgru0 = model_long.predict([x[None,:2000,:], y[None,0,:]])
plotpredictionlong2(y[:2000], ypred_cumsumgru0, frequency, trainwindow, 
                    nrows=nrows, ncols=ncols, ylim=[0-0.05,1+0.05])
x_ext = np.concatenate((x,x),axis=0)
ypred_cumsumgru = model_long.predict([x_ext[None,:,:], y[None,0,:]])
plotpredictionlong2(y_ext, ypred_cumsumgru, frequency, trainwindow, 
                    nrows=nrows, ncols=ncols, ylim=[y_ext.min()-0.05,1+0.05])